In [1]:
import cv2
import numpy as np
from cffi.model import pointer_cache


# 颜色分隔复现

In [ ]:
img=cv2.imread('img_1.png')
cv2.imshow('img',img)
cv2.waitKey(0)

In [ ]:
img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
img_h,img_s,img_v=cv2.split(img_hsv)
# 
mask_h_1 = cv2.inRange(img_h,160,180)
mask_h_2 = cv2.inRange(img_h,0,10)
mask_h = cv2.bitwise_or(mask_h_1,mask_h_2)

mask_s = cv2.inRange(img_s,90,200)

mask=cv2.bitwise_and(mask_h,mask_s)

In [ ]:
img_out = cv2.bitwise_and(img,img,mask=mask)
cv2.imshow('img',img_out)
cv2.imwrite('img_out.png',img_out)

# 最小框选与多边形拟合

In [17]:
img2 = cv2.imread('img.png')
cv2.imshow('img2',img2)


In [18]:
matrix_low = np.array([200,200,200])
matrix_high = np.array([255,255,255])
mask = cv2.inRange(img2,matrix_low,matrix_high)
img_out = cv2.bitwise_and(img2,img2,mask=mask)
cv2.imshow('img2',img_out)

In [ ]:
gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
binary = cv2.bitwise_and(binary, mask)  
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#draw = cv2.drawContours(img2, contours, -1, (0, 255, 0), 2)
#cv2.imshow('img2',draw)
draw_img=np.copy(img2)

for cnt in contours:
    moments = cv2.moments(cnt)
    print(cnt)
    area = cv2.contourArea(cnt)
    if area>100:
        approx = cv2.approxPolyDP(cnt, 0.03*cv2.arcLength(cnt, True), True)
        draw_img = cv2.drawContours(draw_img,[approx],0,(0,0,255),3)
        


# 6. 显示所有结果
cv2.imshow('img2', img_out)
cv2.imshow('Binary', binary)
cv2.imshow('Contours', draw_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[620 669]]]
[[[558 467]]

 [[558 468]]

 [[558 469]]

 [[558 470]]

 [[558 471]]

 [[558 472]]

 [[557 473]]

 [[557 474]]

 [[557 475]]

 [[557 476]]

 [[557 477]]

 [[557 478]]

 [[557 479]]

 [[557 480]]

 [[557 481]]

 [[557 482]]

 [[557 483]]

 [[557 484]]

 [[557 485]]

 [[557 486]]

 [[557 487]]

 [[557 488]]

 [[557 489]]

 [[556 490]]

 [[556 491]]

 [[556 492]]

 [[556 493]]

 [[556 494]]

 [[556 495]]

 [[556 496]]

 [[556 497]]

 [[556 498]]

 [[556 499]]

 [[556 500]]

 [[556 501]]

 [[556 502]]

 [[556 503]]

 [[556 504]]

 [[556 505]]

 [[555 506]]

 [[556 507]]

 [[556 508]]

 [[555 509]]

 [[555 510]]

 [[555 511]]

 [[555 512]]

 [[555 513]]

 [[555 514]]

 [[554 515]]

 [[553 516]]

 [[552 516]]

 [[551 516]]

 [[550 516]]

 [[549 516]]

 [[548 516]]

 [[547 516]]

 [[546 516]]

 [[545 516]]

 [[544 516]]

 [[543 515]]

 [[542 515]]

 [[541 515]]

 [[540 515]]

 [[539 515]]

 [[538 515]]

 [[537 515]]

 [[536 515]]

 [[535 515]]

 [[534 515]]

 [[533 515]]

 [[532

# 视频流多边形跟踪

In [20]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
while True:
    ret, img = cap.read()
    if not ret:
        print("无法获取图像")
        break   
    cv2.imshow("rear",img)
    img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    img_h,img_s,img_v=cv2.split(img_hsv)
# 
    mask_h_1 = cv2.inRange(img_h,175,180)
    mask_h_2 = cv2.inRange(img_h,0,5)
    mask_h = cv2.bitwise_or(mask_h_1,mask_h_2)

    mask_s = cv2.inRange(img_s,90,200)

    mask=cv2.bitwise_and(mask_h,mask_s)
    img_out = cv2.bitwise_and(img,img,mask=mask)
    contours,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    draw_img=np.copy(img)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>1000:
            approx = cv2.approxPolyN(cnt,8, True)
            draw_img = cv2.drawContours(draw_img,[approx],0,(0,255,0),3)
    
    cv2.imshow('img',draw_img)
    key = cv2.waitKey(1)
    if key == 27:  # ESC键退出
        break
cap.release()
cv2.destroyAllWindows()

# 演示结果见视频追踪.mp4

# 物体位置姿态结算

In [ ]:
import cv2
import numpy as np

In [54]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

object_point=np.array([
    [0,  0, 0],
    [0, 13, 0],
    [13,13, 0],
    [13, 0, 0]
],dtype=np.float32)

camera_matrix = np.array([
    [484.3672, 0.0000, 321.0790],
    [0.0000, 484.0169, 237.0419],
    [0.0000, 0.0000, 1.0000]
], dtype=np.float32)

dist_coeffs = np.array([-0.0709, 0.1093, 0.0000, 0.0000], dtype=np.float32)


while True:
    ret, img = cap.read()
    if not ret:
        print("视频获取失败")
        break
    cv2.imshow("rear",img)
    img_hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    img_h,img_s,img_v=cv2.split(img_hsv)

    mask_h_1 = cv2.inRange(img_h,165,180)
    mask_h_2 = cv2.inRange(img_h,0,5)
    mask_h = cv2.bitwise_or(mask_h_1,mask_h_2)

    mask_s = cv2.inRange(img_s,90,200)

    mask=cv2.bitwise_and(mask_h,mask_s)
    img_out = cv2.bitwise_and(img,img,mask=mask)
    cv2.imshow('img_1',img_out)
    contours,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    draw_img=np.copy(img)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>1000:
            rect = np.zeros((4,2),dtype = np.float32)
            M = cv2.moments(cnt)
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        
            area = cv2.contourArea(cnt)
            approx = cv2.approxPolyN(cnt,4, True)
            draw_img = cv2.drawContours(draw_img,[approx],0,(0,255,0),3)
            vertices = approx.reshape(4,2)
            angles = np.arctan2(vertices[:,1] - cy, vertices[:,0] - cx)
            sorted_indices = np.argsort(angles)
            rect = vertices[sorted_indices]
            
            
            cv2.putText(draw_img,f"YS:{rect[0]}",(10,100),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0),3)
            cv2.putText(draw_img,f"YX:{rect[1]}",(10,20),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0),3)
            cv2.putText(draw_img,f"ZX:{rect[2]}",(10,40),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0),3)
            cv2.putText(draw_img,f"ZS:{rect[3]}",(10,70),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0),3)
            
            
            for point in rect:
                cv2.circle(draw_img,(point[0],point[1]),1,(0,0,255),3)
            rect = rect.reshape(4, 1, 2)   
            rect = rect.astype(np.float32)
            _,rvec,tvec = cv2.solvePnP(object_point,rect, camera_matrix, dist_coeffs)
            
            project_axis_points, _ = cv2.projectPoints(
                            object_point, 
                            rvec, 
                            tvec, 
                            camera_matrix, 
                            dist_coeffs
            )

            project_axis_points = project_axis_points.reshape(4, 2).astype(int)

            axis_length = 5.0  
            axis_points = np.array([
                        [0, 0, 0],          # 原点
                        [axis_length, 0, 0], # X轴
                        [0, axis_length, 0], # Y轴
                        [0, 0, axis_length]  # Z轴
            ], dtype=np.float32)

            projected_axis_points, _ = cv2.projectPoints(
                                axis_points, 
                                rvec, 
                                tvec, 
                                camera_matrix, 
                                dist_coeffs
            )

            projected_axis_points = projected_axis_points.reshape(4, 2).astype(int)

            origin = tuple(projected_axis_points[0])
            x_axis = tuple(projected_axis_points[1])
            y_axis = tuple(projected_axis_points[2])
            z_axis = tuple(projected_axis_points[3])

            cv2.line(draw_img, origin, x_axis, (0, 0, 255), 2)
            cv2.line(draw_img, origin, y_axis, (0, 255, 0), 2)
            cv2.line(draw_img, origin, z_axis, (255, 0, 0), 2)

            cv2.putText(draw_img, "X", x_axis, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(draw_img, "Y", y_axis, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(draw_img, "Z", z_axis, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
    cv2.imshow('img',draw_img)
    key = cv2.waitKey(1)
    if key == 27:  # ESC键退出
        break
cap.release()
cv2.destroyAllWindows()
